In [ ]:
from utils import *
perf = np.array([1, 2, np.nan])

In [1]:
name = 'Airfoils2D'
airfoilpath = '/work3/s212645/DiffusionAirfoil/Airfoils/'

from utils import *
v = 5.7
mass = 0.32
area = 0.2254
d = 0.155
thickness = 0.058
reynolds = reynolds_pipe(velocity=v, diameter=d)
_cl = mode_cl(velocity=v, area=area, mass = mass)
af = np.loadtxt('BETTER/20150114-50 +2 d.dat', skiprows=1)
af = interpolate(af, 256, 3)
vs = np.linspace(5.5, 6, 3)
mass = 0.32
area = 0.2254
d = 0.155
thickness = 0.058
perf_bl = type2_simu(af, mass, d, area)
print(perf_bl)

simlog = f'results/{name}_{int(reynolds)}_{_cl:.2f}_simlog.txt'
simperf = f'results/{name}_{int(reynolds)}_{_cl:.2f}_simperf.log'
try:
    lines = []
    with open(simperf, 'r') as fin:
        for line in fin:
            lines.append(line)
    perf_bl = float(lines[-1].split('perf: ')[-1].split(', ')[0])
    print(perf_bl)
except:
    pass

try:
    log = np.loadtxt(simlog)
    i = int(log[0])
    k = int(log[1])
    m = int(log[2])
except:
    m = 0
    i = 0
    k = 0


print(f'i: {i}, k: {k}, m: {m}')
while 1:
    num = str(i).zfill(3)
    airfoils = np.load(airfoilpath+num+'.npy')
    airfoils = delete_intersect(airfoils)
    while k < airfoils.shape[0]:
        airfoil = airfoils[k,:,:]
        airfoil = derotate(airfoil)
        airfoil = Normalize(airfoil)
        airfoil = interpolate(airfoil, 400, 3)
        xhat, yhat = savgol_filter((airfoil[:,0], airfoil[:,1]), 10, 3)
        airfoil[:,0] = xhat
        airfoil[:,1] = yhat
        airfoil[:,1] = airfoil[:,1] * thickness / cal_thickness(airfoil)
        airfoil = interpolate(airfoil, 300, 3)
        if detect_intersect(airfoil):
            perf = np.nan
        else:
            try:
                perf = type2_simu(airfoil, mass, d, area)
                successful = True
                if not np.isnan(perf):
                    print(f'perf: {perf:4f}')
            except Exception as e:
                perf = np.nan
                print(e)
        if np.isnan(perf):
            pass
        elif perf > perf_bl:
            perf_bl = perf
            mm = str(m).zfill(3)
            af_name = f'{name}_{int(reynolds)}_{_cl:.2f}_{int(mm)}'
            np.savetxt(f'samplesP5B/{af_name}.dat', airfoil, header=f'{af_name}', comments="")
            f = open(simperf, 'a')
            f.write(f'perf: {perf}, m: {mm}, path: samplesP5B/{af_name}.dat\n')
            f.close()
            m += 1
            del f
        k += 1
        log = np.array([i, k, m])
        np.savetxt(simlog, log)
        del airfoil
        gc.collect()
    k = 0
    i += 1
    del airfoils
    gc.collect()

44.258796788783854
45.84985865991439
i: 0, k: 829, m: 3


/zhome/02/b/164706/Master_Courses/2023_Fall/DiffusionAirfoil/utils.py:74: RuntimeWarning: Mean of empty slice.
  perfs = np.array(perfs).mean()
/zhome/02/b/164706/miniconda3/envs/pytorch/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


perf: 42.599807
perf: 27.900119
perf: 33.031955
perf: 37.257776
perf: 38.307549
perf: 30.913212
perf: 41.653914
perf: 22.467628
perf: 29.873517
perf: 20.875187
perf: 35.244171
perf: 25.060994
perf: 31.347245
perf: 24.697428
perf: 40.718216
perf: 32.415965
perf: 25.969842
perf: 21.047007
perf: 34.062676
perf: 18.006308
perf: 20.646637
perf: 26.433846
perf: 40.106921
perf: 35.333372
perf: 30.663862
perf: 24.039254
perf: 19.141977
perf: 38.247357
perf: 20.754419
perf: 19.551024
perf: 30.984852
perf: 29.507537
perf: 44.387784
perf: 23.840243
perf: 18.558083
perf: 28.084336
perf: 30.223024
perf: 18.715580
perf: 33.149059
perf: 23.773639
perf: 30.308505
perf: 44.396288
perf: 21.693978
perf: 26.977042
perf: 30.772802
perf: 33.280388
perf: 37.353310
perf: 17.377524
perf: 21.110562
perf: 36.636909
perf: 38.888595
perf: 20.193406
perf: 26.693768
perf: 27.597208
perf: 32.927624
perf: 37.139544
perf: 23.385062
perf: 30.634336
perf: 41.406518
perf: 41.160184
perf: 34.649534
perf: 39.736153
perf: 25

In [23]:
from utils import *
def type2_simu(af, mass, diameter, area):
    g = 9.806
    density = 1.225
    viscosity = 1.5*10**(-5)
    re_sqrtcl = np.sqrt(2 * mass * g / density / area) * diameter * density / viscosity
    cl_list = np.linspace(0.6, 0.7, 6)
    perfs = []
    for cl in cl_list:
        re = re_sqrtcl / np.sqrt(cl)
        perf,_,_ = evalperf(af, cl, re)
        if not np.isnan(perf):
            perfs.append(perf)
    perfs = np.array(perfs).mean()
    return perfs

mass = 0.32
area = 0.2254
d = 0.155
af = np.loadtxt('BETTER/20150114-50 +2 d.dat', skiprows=1)
af = interpolate(af, 256, 3)
perf_bl = type2_simu(af, mass, d, area)
perf_bl

44.07490150108495

For steady level flight the lift equation is
$L=W=\frac{1}{2} \rho V^2 * S * C_L$
And Reynold's number is
$\text{Re}= \frac{Vl\rho}{\mu}$
Rearranging the lift equation for V gives
$V=\sqrt\frac{2W}{\rho S c_L}$
Using this V in the Reynold's number gives
$\text{Re}=\sqrt{\frac{2W}{\rho S c_L}}\frac{l \rho}{\mu}$
Therefore
$\text{Re}\sqrt{c_L}=\sqrt{\frac{2W}{\rho S}}\frac{l \rho}{\mu}$

In [27]:
from utils import *
import re
import os
dat = re.compile('.dat')
# dat = re.compile('airfoil.*\.dat')
root = 'samplesP5B/'

airfoils = {}
for path, dir, files in os.walk(root):
    for file in files:
        if dat.search(file) is not None:
            airfoils['{}{}'.format(path,file)] = file.split('.dat')[0]
            # airfoils.append('{}{}'.format(path,file))

Rbl = 1
files.sort()
for file in airfoils.keys():
    points = file
    name = airfoils[file]
    airfoil = np.loadtxt(points, skiprows=1)
    perf = type2_simu(airfoil, mass, d, area)
    if perf < 44.07490150108495:
        os.remove(points)
        print(points, name)

samplesP5B/DiffusionAirfoilTransform_63291_0.63_1.dat DiffusionAirfoilTransform_63291_0.63_1
samplesP5B/DiffusionAirfoil1DTransform_72152_0.70_1.dat DiffusionAirfoil1DTransform_72152_0.70_1
samplesP5B/Airfoils1D_72152_0.70_1.dat Airfoils1D_72152_0.70_1
samplesP5B/Airfoils1D_72152_0.70_0.dat Airfoils1D_72152_0.70_0
samplesP5B/DiffusionAirfoilTransform_72152_0.70_1.dat DiffusionAirfoilTransform_72152_0.70_1
samplesP5B/DiffusionAirfoilTransform_72152_0.70_3.dat DiffusionAirfoilTransform_72152_0.70_3
samplesP5B/Airfoils1D_72152_0.70_2.dat Airfoils1D_72152_0.70_2
samplesP5B/DiffusionAirfoil1DTransform_72152_0.70_4.dat DiffusionAirfoil1DTransform_72152_0.70_4
samplesP5B/DiffusionAirfoilTransform_72152_0.70_6.dat DiffusionAirfoilTransform_72152_0.70_6
samplesP5B/Airfoils1D_72152_0.70_4.dat Airfoils1D_72152_0.70_4
samplesP5B/DiffusionAirfoil1DTransform_72152_0.70_2.dat DiffusionAirfoil1DTransform_72152_0.70_2
samplesP5B/DiffusionAirfoilTransform_72152_0.70_2.dat DiffusionAirfoilTransform_72152

In [ ]:

mass = 0.32
area = 0.2254
d = 0.155
airfoil = np.loadtxt('samplesP5B/Airfoils2D_72152_0.70_2.dat', skiprows=1)
perf = type2_simu(airfoil, mass, d, area)
perf